# Algorytmika i matematyka uczenia maszynowego 
## Laboratorium 11 - macierz odwrotna i wyznacznik macierzy


### Zadanie 1 - szyfr Hilla

Zaimplementuj dwie funkcje do kodowania i dekodowania wiadomości za pomocą szyfru [Hilla](https://pl.wikipedia.org/wiki/Szyfr_Hilla):
* `encrypt(t, K)` - szyfruje tekst $t$ za pomocą klucza (macierzy) $K$.
* `decrypt(s, K)` - deszyfruje kod $s$ za pomocą klucza (macierzy) $K$.

#### Kodowanie

W systemie kodowania ASCII litery A - Z zapisane są jako liczby z zakresu 65 - 90. Aby zaszyfrować tekst za pomocą klucza $K$ (macierz o wymiarach $m$ x $m$), należy zapisać znaki tekstu w postaci macierzy o wymiarach $m$ x $n$, a następnie wykonać następujące operacje:

1. Utworzyć macierz szyfrującą $K$, której wyznacznik wynosi $det(K) = 1$.
> Uwaga: jest to odstępstwo od oryginalnego algorytmu mające na celu uproszczenie przykładu.
> Macierz taką można utworzyć z macierzy jednostkowej (`np.identity`), korzystając z operacji elementarnych np. dodając do jednego wiersza macierzy inny wiersz pomnożony przez skalar.
1. Zamienić litery tekstu $t$ o długości $h$ na wektor liczb.
1. Dopełnić (_padding_) zerami, aby można było wykonać kolejny krok.
1. Przekształcić na macierz $X$ o wymiarach ($m$ x $n$), gdzie $n = \lceil \frac{h}{m} \rceil$ (możesz użyć funkcji `reshape`).
1. Wykonać operację $S = (KX)$.
1. Skonwertować macierz $S$ na wektor (możesz użyć funkcji `flatten`) i zwrócić szyfrogram $s$ (zaszyfrowany tekst).


> Uwaga 1: Przedstawiony algorytm jest uproszczonym algorytmem, posiadającym ograniczenie $det(K)=1$, które można pominąć, ale wtedy należy do macierzy kodującej wyznaczyć macierz odwrotną modulo 26 (liczba znaków A-Z, ale może być dowolna inna). Podobnie, należy macierz $S$ zamienić na modulo 26. **Istotne**: W tym przypadku należy pamiętać, że wyznacznik macierzy szyfrującej $det(K)$ nie może posiadać wspólnego dzielnika z liczbą 26 (czyli obie liczby muszą być względnie pierwsze). Dlaczego? Bo w przeciwnym wypadku nie istnieje liczba odwrotna do $det(K) \textit{ mod } 26$.

> Uwaga 2: ciąg $s$ może zawierać niedrukowalne znaki. Jeżeli chcesz tego uniknąć możesz np. zmapować znaki (65-90) do zakresu 0-25. Następnie przy wyświetlaniu przeprowadzić operację w drugą stronę.


#### Dekodowanie

Aby rozszyfrować zaszyfrowaną wiadomość $s$, należy:

1. Zamienić ciąg $s$ na macierz $S$.
1. Obliczyć macierz odwrotną $K^{-1}$ (funkcja `np.linalg.inv`).
1. Rozszyfrować wiadomość wykonując operację $W = K^{-1} S$.
1. Skonwertować wiadomość $W$ na ciąg tekstowy $w$.


**Uwaga**: Jest to tylko laboratoryjny przykład na zastosowanie operacji macierzowych. Przedstawione rozwiązanie nie jest bezpieczne.

In [84]:
from numpy import ndarray, array, identity, append
from numpy.linalg import inv, det
import numpy as np


def encrypt(t: str, k: int) -> tuple:
    code = identity(k) * 10
    text = array([ord(letter) for letter in list(t)])

    if text.shape[0] % k > 0:
        for _ in range(k - text.shape[0] % k):
            text = append(text, array([0]))

    text = text.reshape((k, int(text.shape[0] / k)))
    s = code @ text

    return ''.join([chr(txt) for txt in s.flatten()[:len(t)].astype(int)]), code


def decrypt(s: str, k: ndarray) -> str:
    text = array([ord(ss) for ss in list(s)])

    if text.shape[0] % k.shape[0] > 0:
        for _ in range(k.shape[0] - text.shape[0] % k.shape[0]):
            text = append(text, array([0]))

    text = text.reshape((k.shape[0], int(text.shape[0] / k.shape[0])))
    decoded = inv(k) @ text

    return ''.join([chr(txt) for txt in decoded.flatten()[:len(s)].astype(int)])

In [85]:
text = 'ala ma kota'

In [86]:
t_code, code = encrypt(text, 3)
t_code

'ϊиϊŀтϊŀЮі҈ϊ'

In [87]:
decrypt(t_code, code)

'ala ma kota'

### Zadanie 2

Zaimplementować funkcję, która przyjmuje macierz kwadratową jako argument i zwraca jej wyznacznik obliczony zgodnie ze wzorem [Leibniza](https://en.wikipedia.org/wiki/Leibniz_formula_for_determinants) (_definicja permutacyjna_) i porównaj z wynikiem gotowej funkcji z biblioteki numpy `np.linalg.det`.

$$
\text{det}(A) = \sum_{\sigma \in S_n}\left(\text{sgn}(\sigma)\prod_{i=0}^{n-1}a_{i, \sigma(i)}\right)
$$

, gdzie:

* $S_n$ - [grupa permutacji](https://en.wikipedia.org/wiki/Symmetric_group) (dla macierzy 3x3 będą to permutacje ze zbioru {0, 1, 2})
* $\text{sgn}$ - jest to symbol "+", "-" w zależny od [parzystości permutacji](https://en.wikipedia.org/wiki/Parity_of_a_permutation). Np. dla permutacji `[1, 2, 0]` będzie to "+" (trzeba wykonać dwie operacje - zamienić `0` z `1` a później `0` z `2`, a dla permutacji `[0, 2, 1]` będzie "-" ponieważ wystarczy jedna operacja (zamiana `1` z `2`).
* $\sigma$ - permutacja (element z grupy permutacji $S_n$)

##### Przykład dla macierzy 3x3

| $\sigma$ | $\text{sgn}$ | $\text{sgn}(\sigma)\prod_{i=0}^{n-1}a_{i, \sigma(i)}$ |
| :---     | :---         | ---: |
| 1, 2, 3  | +            | $+a_{1,1}a_{2,2}a_{3,3}$ |
| 1, 3, 2  | -            | $-a_{1,1}a_{2,3}a_{3,2}$ |
| 3, 1, 2  | +            | $+a_{1,3}a_{2,1}a_{3,2}$ |
| 3, 2, 1  | -            | $-a_{1,3}a_{2,2}a_{3,1}$ |
| 2, 3, 1  | +            | $+a_{1,2}a_{2,3}a_{3,1}$ |
| 2, 1, 3  | -            | $-a_{1,2}a_{2,1}a_{3,3}$ |

$\text{det}(A) = a_{1,1}a_{2,2}a_{3,3} - a_{1,1}a_{2,3}a_{3,2} + a_{1,3}a_{2,1}a_{3,2} - a_{1,3}a_{2,2}a_{3,1} + a_{1,2}a_{2,3}a_{3,1} - a_{1,2}a_{2,1}a_{3,3}$

> Uwaga 1: Aby sprawdzić parzystość permutacji możesz użyć funkcji `parity` z biblioteki [`sympy`](https://docs.sympy.org/latest/modules/combinatorics/permutations.html#sympy.combinatorics.permutations.Permutation.parity). Przykład: 
    ```from sympy.combinatorics import Permutation
Permutation([0, 2, 1]).parity()
    ```
    
> Uwaga 2: W celu wygenerowania permutacji możesz użyć funkcji `permutations` z modułu [`itertools`](https://docs.python.org/3/library/itertools.html#itertools.permutations)

> Uwaga 3: Pamiętaj, że w numpy porównywanie liczb zmiennoprzecinkowych wykonuje się za pomocą funkcji [`allclose`](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html)

In [115]:
from numpy.random import randint
from numpy import prod, allclose
from itertools import permutations
from sympy.combinatorics import Permutation


def determinant(matrix: ndarray) -> float:
    idx = list(range(matrix.shape[0]))
    result = 0.0

    for perm in permutations(idx):
        val = -1 if Permutation(perm).parity() == 1 else 1
        result += val * prod([matrix[i, perm[i]] for i in idx])

    return result


mat = randint(low=0, high=10, size=(4, 4))
print(allclose(determinant(mat), det(mat)))

True
